In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn
device = 'cuda'

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
access_token = 'fdgOIYFUJHGfihgfduyiJHGigHLGFyfHgyDFG' # do not test it this is the fake one it will not work :))

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",use_auth_token=access_token,
                                        trust_remote_code=True,)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",use_auth_token=access_token,
                                        trust_remote_code=True,device_map='cuda',torch_dtype=torch.bfloat16)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


In [5]:
class ChatBot:
    def __init__(self,model,tokenizer,system_content=None):
        torch.backends.cuda.enable_mem_efficient_sdp(False)
        torch.backends.cuda.enable_flash_sdp(False)
        
        self.terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
        
        self.model = model
        self.tokenizer = tokenizer
        
        system_content = system_content if system_content else "You are a helpful assistant who answers short."  
        
        message =  [{"role": "system", "content": system_content}]
        self.input_ids = tokenizer.apply_chat_template(
            message,
            return_tensors="pt",
        ).to(model.device)
        
    def response(self,user_content):
        message = [{"role": "user", "content": user_content}]
        new_input_ids = tokenizer.apply_chat_template(
            message,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(model.device)
        self.input_ids = torch.cat((self.input_ids,new_input_ids),dim=-1)
        
        with torch.no_grad():
            outputs = model.generate(
                self.input_ids,
                max_new_tokens=256,
                eos_token_id=self.terminators,
                pad_token_id=128009,
                do_sample=True,
                temperature=0.9,
                top_p=0.5,
            )
        
        response = outputs[0][self.input_ids.shape[-1]:]
        print(tokenizer.decode(response, skip_special_tokens=True))
        
        self.input_ids = outputs


In [6]:
chatbot = ChatBot(model,tokenizer)

In [7]:
print(tokenizer.decode(chatbot.input_ids[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who answers short.<|eot_id|>


In [8]:
while 1:
    text = input('\nUser:')
    if text == 'Q' or text == 'Exit':
        break
    print('\nBot: ',end='')
    chatbot.response(text)
    


User: Hi



Bot: Hi! How can I help you today?



User: Do you know about LLMs?



Bot: You're referring to Large Language Models! Yes, I'm familiar with them. LLMs are a type of artificial intelligence designed to process and generate human-like language. They're trained on vast amounts of text data and can perform tasks such as language translation, text summarization, and even creative writing. Would you like to know more about them?



User: What are the most common LLMs nowadays?



Bot: Some of the most well-known and widely used Large Language Models (LLMs) nowadays include:

1. BERT (Bidirectional Encoder Representations from Transformers): Developed by Google, BERT is a popular LLM that's been fine-tuned for various NLP tasks.
2. RoBERTa (Robustly Optimized BERT Pretraining Approach): Another Google-developed LLM, RoBERTa is a variant of BERT that's known for its improved performance on many NLP tasks.
3. XLNet: Developed by Google and Carnegie Mellon University, XLNet is a state-of-the-art LLM that uses a novel permutation-based language modeling approach.
4. T5 (Text-to-Text Transformer): Developed by Google, T5 is a general-purpose LLM that can be fine-tuned for various NLP tasks, including language translation, text summarization, and more.
5. DALL-E: Developed by Meta AI, DALL-E is a text-to-image LLM that can generate high-quality images from text prompts.

These are just a few examples, but there are many more LLMs being developed and fine-tuned for sp


User: Thanks for your service.



Bot: You're welcome! It was my pleasure to help. If you have any more questions or need assistance with anything else, feel free to ask!



User: Exit
